In [ ]:
# scanpy
import pandas as pd
import numpy as np
import scanpy as sc
import matplotlib.pyplot as plt
import seaborn as sns
import anndata as ad
import scanpy.external as sce
import os
import re
import warnings
warnings.filterwarnings("ignore")
sc.logging.print_header()
sc.__version__
ad.__version__
ir.__version__
import tarfile
import warnings
from glob import glob
import anndata
import matplotlib.pyplot as plt
import muon as mu
import pandas as pd
import scanpy as sc
import scirpy as ir

#### 1. input data and merge GEX TCR and BCR

In [ ]:
## 1. read in raw GEX matrix & TCR & BCR By Sample
sample_list=["cart","day0","day123","day180","day361","day6","day60","day91"]
tcr_list=["cart_tcr","day0_tcr","day123_tcr","day180_tcr","day361_tcr","day6_tcr","day60_tcr","day91_tcr"] 
print(sample_list)

for i in range(len(sample_list)):
    input_name = sample_list[i]
    sample_name = sample_list[i]
    t = tcr_list[i]
    indir = r'/path/Cellranger/'
    sampledir = str(input_name)+'/outs/per_sample_outs/'+str(input_name)+'/count/sample_filtered_feature_bc_matrix/'
    sampledir_tcr = str(input_name)+'/outs/per_sample_outs/'+str(input_name)+'/vdj_t/filtered_contig_annotations.csv'
    exec(sample_name + '= %s' % 'sc.read_10x_mtx(os.path.join(indir,sampledir),cache=True)')
    exec(t + '= %s' % 'ir.io.read_10x_vdj(os.path.join(indir,sampledir_tcr))')
    print(t)
    print(sample_name)

# BCR
sample_list_bcr = ['day0','day123','day180','day361','day60','day91']
bcr_list=["day0_bcr","day123_bcr","day180_bcr","day361_bcr","day60_bcr","day91_bcr"] # "cart_bcr","day6_bcr",
for i in range(len(sample_list_bcr)):
    input_name = sample_list_bcr[i]
    sample_name = sample_list_bcr[i]
    b = bcr_list[i]
    indir = r'/path/Cellranger/'
    sampledir_bcr = str(input_name)+'/outs/per_sample_outs/'+str(input_name)+'/vdj_b/filtered_contig_annotations.csv'
    exec(b + '= %s' % 'ir.io.read_10x_vdj(os.path.join(indir,sampledir_bcr))')
    print(b)
    print(sample_name)

#### 2. QC TCR and BCR and merge AIRR information

In [ ]:
## 1.1 TCR and BCR
# TCR Quality Control
tcr_list=["cart_tcr","day0_tcr","day123_tcr","day180_tcr","day361_tcr","day6_tcr","day60_tcr","day91_tcr"] 
for i in tcr_list:
    sample = locals()[i]
    ir.pp.index_chains(sample, filter=["productive", "require_junction_aa"])
    ir.tl.chain_qc(sample) 
    sample.obs["Sample"] = str(i).split("_")[0]
    
# BCR Quality Control
bcr_list=["day0_bcr","day123_bcr","day180_bcr","day361_bcr","day60_bcr","day91_bcr"] 
for i in bcr_list:
    sample = locals()[i]
    ir.pp.index_chains(sample, filter=["productive", "require_junction_aa"])
    ir.tl.chain_qc(sample)
    sample.obs["Sample"] = str(i).split("_")[0]

In [ ]:
## 1.2 Map each cell barcode to its respective numeric index (assumes obs_names are unique)
# TCR
sample_list=["cart","day0","day123","day180","day361","day6","day60","day91"]
tcr_list=["cart_tcr","day0_tcr","day123_tcr","day180_tcr","day361_tcr","day6_tcr","day60_tcr","day91_tcr"] 
dict_tcr = {"cart":"cart_tcr","day0":"day0_tcr","day123":"day123_tcr","day180":"day180_tcr","day361":"day361_tcr","day6":"day6_tcr","day60":"day60_tcr","day91":"day91_tcr"}
TCR_obs_list= []
for i,j in dict_tcr.items():
    sample_gex = locals()[i]
    sample_tcr = locals()[j]
    barcode2idx = {barcode: i for i, barcode in enumerate(sample_tcr.obs_names)}
    idx = [barcode2idx.get(barcode, None) for barcode in sample_gex.obs_names]
    sample_gex.obsm["tcr_airr"] = sample_tcr.obsm["airr"][idx]
    ir.get.airr_context(sample_tcr, "locus", ["VJ_1", "VDJ_1", "VJ_2", "VDJ_2"])
    TCR_obs_df = sample_tcr.obs.copy()
    TCR_obs_list.append(TCR_obs_df)
TCR_obs_merge = pd.concat(TCR_obs_list, ignore_index=False)

# BCR
sample_list=["day0","day123","day180","day361","day60","day91"]
bcr_list=["day0_bcr","day123_bcr","day180_bcr","day361_bcr","day60_bcr","day91_bcr"] 
dict_bcr = {"day0":"day0_bcr","day123":"day123_bcr","day180":"day180_bcr","day361":"day361_bcr","day60":"day60_bcr","day91":"day91_bcr"}
BCR_obs_list= []
for i,j in dict_bcr.items():
    sample_gex = locals()[i]
    sample_bcr = locals()[j]
    barcode2idx = {barcode: i for i, barcode in enumerate(sample_bcr.obs_names)}
    idx = [barcode2idx.get(barcode, None) for barcode in sample_gex.obs_names]
    sample_gex.obsm["bcr_airr"] = sample_bcr.obsm["airr"][idx]
    BCR_obs_df = sample_bcr.obs.copy()
    BCR_obs_list.append(BCR_obs_df)
BCR_obs_merge = pd.concat(BCR_obs_list, ignore_index=False)

# concat BCR and TCR
TCR_obs_merge["Barcode_Raw"] = TCR_obs_merge.index
TCR_obs_merge["Barcode_Raw"] = TCR_obs_merge["Barcode_Raw"] + "-"+ TCR_obs_merge["Sample"]
TCR_obs_merge = TCR_obs_merge.reset_index(drop=True)
TCR_obs_merge.columns = TCR_obs_merge.columns + str("_TCR")
#
BCR_obs_merge["Barcode_Raw"] = BCR_obs_merge.index
BCR_obs_merge["Barcode_Raw"] = BCR_obs_merge["Barcode_Raw"] + "-"+ BCR_obs_merge["Sample"]
BCR_obs_merge = BCR_obs_merge.reset_index(drop=True)
BCR_obs_merge.columns = BCR_obs_merge.columns + str("_BCR")
obs_merge = pd.merge(TCR_obs_merge,BCR_obs_merge,left_on="Barcode_Raw_TCR",right_on="Barcode_Raw_BCR",how="outer")
# BC
obs_merge_BC = obs_merge[["Barcode_Raw_TCR","Barcode_Raw_BCR"]]
obs_merge_BC["Barcode_Raw_TCR"] = obs_merge_BC["Barcode_Raw_TCR"].fillna(obs_merge_BC["Barcode_Raw_BCR"])
obs_merge["Barcode_Raw"] = list(obs_merge_BC["Barcode_Raw_TCR"])
# Sample
obs_merge_Sample = obs_merge[["Sample_TCR","Sample_BCR"]]
obs_merge_Sample["Sample_TCR"] = obs_merge_Sample["Sample_TCR"].fillna(obs_merge_Sample["Sample_BCR"])
obs_merge["Sample"] = list(obs_merge_Sample["Sample_TCR"])
obs_merge.to_csv('/path/UCART_SLE_8Samples_AIRR_Scirpy.csv')

##### 2.1. combined TCR and BCR filtered contigs annotation

In [ ]:
# TCR
tcr_list=["cart_tcr","day0_tcr","day123_tcr","day180_tcr","day361_tcr","day6_tcr","day60_tcr","day91_tcr"]
all_tcr_df_list = []
indir = r'/path/Cellranger/'
for i in range(len(tcr_list)):
    t = tcr_list[i]
    input_name = t.replace("_tcr", "")
    sampledir_tcr = os.path.join(str(input_name), 'outs', 'per_sample_outs', str(input_name), 'vdj_t', 'filtered_contig_annotations.csv')
    t_df = pd.read_csv(os.path.join(indir, sampledir_tcr))
    exec('t_df["Sample"] = %s' % 'input_name')
    all_tcr_df_list.append(t_df)

merged_tcr_df = pd.concat(all_tcr_df_list, ignore_index=True)
merged_tcr_df.columns = merged_tcr_df.columns + str("_TCR")
# Sample_batch_dict = ADATA.obs.set_index("Sample")["batch"].to_dict()
# merged_tcr_df["batch"] = merged_tcr_df["Sample_TCR"].str.replace("_tcr", "").map(Sample_batch_dict)
merged_tcr_df["barcode_TCR"] = merged_tcr_df["barcode_TCR"] + "-" + merged_tcr_df["Sample_TCR"]
merged_tcr_df.to_csv("/path/UCART_SLE_8Samples_TCR_filtered_contig_annotations_Raw.csv")


# BCR
bcr_list=["day0_bcr","day123_bcr","day180_bcr","day361_bcr","day60_bcr","day91_bcr"]
all_bcr_df_list = []
indir = r'/path/Cellranger/'
for i in range(len(bcr_list)):
    b = bcr_list[i]
    input_name = b.replace("_bcr", "")
    sampledir_bcr = os.path.join(str(input_name), 'outs', 'per_sample_outs', str(input_name), 'vdj_b', 'filtered_contig_annotations.csv')
    b_df = pd.read_csv(os.path.join(indir, sampledir_bcr))
    exec('b_df["Sample"] = %s' % 'input_name')
    all_bcr_df_list.append(b_df)
  
merged_bcr_df = pd.concat(all_bcr_df_list, ignore_index=True)
merged_bcr_df.columns = merged_bcr_df.columns + str('_BCR')

merged_bcr_df["barcode_BCR"] = merged_bcr_df["barcode_BCR"] + "-" + merged_bcr_df["Sample_BCR"]
merged_bcr_df.to_csv("/path/UCART_SLE_6Samples_BCR_filtered_contig_annotations_Raw.csv")

In [ ]:
# Merge AIRR information for adding to meta.data
obs_merge = pd.read_csv('/path/UCART_SLE_8Samples_AIRR_Scirpy.csv',index_col=0)

#
df_TCR = pd.read_csv("/path/UCART_SLE_8Samples_TCR_filtered_contig_annotations_Raw.csv",index_col=0)
df_BCR = pd.read_csv("/path/UCART_SLE_6Samples_BCR_filtered_contig_annotations_Raw.csv",index_col=0)
df_TCR["AIRR_Type_TCR"] = "TCR"
df_BCR["AIRR_Type_BCR"] = "BCR"
df_TCR_CloneID = df_TCR[["barcode_TCR","raw_clonotype_id_TCR","Sample_TCR","AIRR_Type_TCR"]].drop_duplicates()
df_BCR_CloneID = df_BCR[["barcode_BCR","raw_clonotype_id_BCR","Sample_BCR","AIRR_Type_BCR"]].drop_duplicates()
df_AIRR = pd.merge(df_TCR_CloneID, df_BCR_CloneID, left_on = "barcode_TCR", right_on= "barcode_BCR", how="outer")
# BC
df_AIRR_BC = df_AIRR[["barcode_TCR","barcode_BCR"]]
df_AIRR_BC['barcode_TCR'] = df_AIRR_BC['barcode_TCR'].fillna(df_AIRR_BC['barcode_BCR'])
df_AIRR["barcode"] = list(df_AIRR_BC["barcode_TCR"])
# Sample
df_AIRR_Sample = df_AIRR[["Sample_TCR","Sample_BCR"]]
df_AIRR_Sample['Sample_TCR'] = df_AIRR_Sample['Sample_TCR'].fillna(df_AIRR_Sample['Sample_BCR'])
df_AIRR["Sample"] = list(df_AIRR_Sample["Sample_TCR"])
#
DF = pd.merge(obs_merge, df_AIRR, left_on = "Barcode_Raw", right_on="barcode", how="left")
DF = DF[[col for col in DF.columns if not col.endswith('_y')]]
DF.columns = [col.replace('_x','') for col in DF.columns]
DF.to_csv('/share/home/qinxiaohuan/proj/06.CART_SLE/01.h5ad/UCART_SLE_8Samples_TCRBCR_filtered_contig_annotations_Processed.csv',index=False)

#### 3. QC GEX

In [ ]:
## 2.1 GEX
# 1. drop doublet
sample_list=["cart","day0","day6","day60","day91","day123","day180","day361"]

for i in sample_list:
    sample = locals()[i]
    sce.pp.scrublet(sample)
    outdir = r'/path/01.QC'
    png = str(i)+'_doublet.png'
    out_doublet_png = os.path.join(outdir,png)
    sce.pl.scrublet_score_distribution(sample)
    plt.savefig(out_doublet_png)
    print(i)
    print(sample)

## 3. basic qc
for i in sample_list:
    sample = locals()[i]
    sc.pp.filter_cells(sample, min_genes = 200)
    sc.pp.filter_genes(sample, min_cells = 0)
    sample.var['mt'] = sample.var_names.str.startswith('MT-')
    sc.pp.calculate_qc_metrics(sample,qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)
    #sample=sample[sample.obs.pct_counts_mt < 20, :]
    locals()[i]=sample
    sample
    i

for i in sample_list:
    sample = locals()[i]
    outdir = r'/path/01.QC'
    png = str(i)+'_QC.png'
    out_png = os.path.join(outdir,png)
    sc.pl.violin(sample,keys=["n_genes","total_counts",'pct_counts_mt',"doublet_score"],stripplot=False,jitter=0.4,multi_panel=True,show=False)
    plt.savefig(out_png,bbox_inches='tight')


# 4. filter out cells by Sample
cart
cart = cart[(cart.obs.n_genes > 500) & (cart.obs.n_genes < 8000) & (cart.obs.total_counts > 1000) & (cart.obs.total_counts < 50000) & (cart.obs.pct_counts_mt < 20),:]
cart = cart[cart.obs["predicted_doublet"]==False]
day0
day0 = day0[(day0.obs.n_genes > 500) & (day0.obs.n_genes < 6000) & (day0.obs.total_counts > 1000) & (day0.obs.total_counts < 50000) & (day0.obs.pct_counts_mt < 20),:]
day0 = day0[day0.obs["predicted_doublet"]==False]
day6
day6 = day6[(day6.obs.n_genes > 500) & (day6.obs.n_genes < 7000) & (day6.obs.total_counts > 1000) & (day6.obs.total_counts < 50000) & (day6.obs.pct_counts_mt < 20),:]
day6 = day6[day6.obs["predicted_doublet"]==False]
day60
day60 = day60[(day60.obs.n_genes > 500) & (day60.obs.n_genes < 6500) & (day60.obs.total_counts > 1000) & (day60.obs.total_counts < 50000) & (day60.obs.pct_counts_mt < 20),:]
day60 = day60[day60.obs["predicted_doublet"]==False]
day91
day91 = day91[(day91.obs.n_genes > 500) & (day91.obs.n_genes < 5500) & (day91.obs.total_counts > 1000) & (day91.obs.total_counts < 50000) & (day91.obs.pct_counts_mt < 20),:]
day91 = day91[day91.obs["predicted_doublet"]==False]
day123
day123 = day123[(day123.obs.n_genes > 500) & (day123.obs.n_genes < 8500) & (day123.obs.total_counts > 1000) & (day123.obs.total_counts < 50000) & (day123.obs.pct_counts_mt < 20),:]
day123 = day123[day123.obs["predicted_doublet"]==False]
day180
day180 = day180[(day180.obs.n_genes > 500) & (day180.obs.n_genes < 7000) & (day180.obs.total_counts > 1000) & (day180.obs.total_counts < 50000) & (day180.obs.pct_counts_mt < 20),:]
day180 = day180[day180.obs["predicted_doublet"]==False]
day361
day361 = day361[(day361.obs.n_genes > 500) & (day361.obs.n_genes < 6000) & (day361.obs.total_counts > 1000) & (day361.obs.total_counts < 50000) & (day361.obs.pct_counts_mt < 20),:]
day361 = day361[day361.obs["predicted_doublet"]==False]


# 5. concate all sample
cart.obs["Sample"] = "cart"
day0.obs["Sample"] = "preTreat"
day6.obs["Sample"] = "day6"
day60.obs["Sample"] = "day60"
day91.obs["Sample"] = "day91"
day123.obs["Sample"] = "day123"
day180.obs["Sample"] = "day180"
day361.obs["Sample"] = "day361"
adata = sc.AnnData.concatenate(cart, day0, day6, day60, day91, day123, day180, day361, join='outer', index_unique='-', batch_key='Sample', batch_categories=['cart', 'day0', 'day6', 'day60', 'day91', 'day123', 'day180', 'day361'], fill_value=0)

adata = sc.read("/path/UCART_SLE_8Samples_RAW.h5ad")
adata.obs["Sample"] = adata.obs["Sample"].replace({"day0":"preTreat"})
sc.pl.violin(adata, keys=["n_genes","total_counts",'pct_counts_mt',"doublet_score"],stripplot=False,inner="box",multi_panel=True,show=False,groupby="Sample",rotation=90)
plt.savefig("/path/UCART_8Samples_QC_Raw.pdf",bbox_inches='tight')
adata.write("/path/UCART_SLE_8Samples_RAW.h5ad")


adata = sc.read("/path/UCART_SLE_8Samples_QC.h5ad")

# 6. Add TCR & BCR info to adata.obs
df_AIRR = pd.read_csv("/path/UCART_SLE_8Samples_TCRBCR_filtered_contig_annotations_Processed.csv")
df_meta =  adata.obs
df_meta["BC"] = list(df_meta.index)
df_meta = pd.merge(df_meta, df_AIRR, left_on='BC', right_on='barcode', how='left')
df_meta = df_meta[[col for col in df_meta.columns if col not in ["barcode", "Sample_y"]]]
print(list(adata.obs.index)==df_meta["BC"].tolist())
df_meta.index = list(df_meta["BC"])
df_meta.columns = [col.replace('_x','') for col in df_meta.columns]
adata.obs = df_meta
adata.write_h5ad("/path/UCART_SLE_8Samples_QC.h5ad")

#### Add CAR info to adata.obs

In [ ]:
adata = sc.read("/path/UCART_SLE_8Samples_QC.h5ad")
adata_car = sc.read("/path/UCART_SLE_8Samples_CAR_Raw.h5ad")
df = adata_car.to_df()
df["barcode"] = df.index
df1 = adata.obs
DF = pd.merge(df1, df, left_on='BC', right_on='barcode', how='left')
print(list(adata.obs.index)==DF["BC"].tolist())
adata.obs["CAR"] = list(DF["CAR"])
adata.obs["CARExp"] = adata.obs["CAR"]
adata.obs["CAR"] = adata.obs["CAR"].apply(lambda x: "CARpos" if x > 0 else "CARneg")
df = adata.obs
df.to_csv("/path/UCART_SLE_8Samples_QC_MetaData.csv")
adata.write_h5ad("/path/UCART_SLE_8Samples_QC.h5ad")

In [ ]:
adata = sc.read("/path/UCART_SLE_8Samples_QC.h5ad")
df = adata.obs[["AIRR_Type_BCR","AIRR_Type_TCR"]]
df["AIRR_Type_BCR"] = pd.Categorical(df["AIRR_Type_BCR"], categories=['TCR', 'BCR'], ordered=True)
df["AIRR_Type_TCR"] = pd.Categorical(df["AIRR_Type_TCR"], categories=['TCR', 'BCR'], ordered=True)
df["AIRR_Type"] = df["AIRR_Type_BCR"]
df["AIRR_Type"] = df["AIRR_Type"].fillna(df["AIRR_Type_TCR"])
adata.obs["AIRR_Type"] = df["AIRR_Type"]
df = adata.obs
df.to_csv("/path/UCART_SLE_8Samples_QC_MetaData.csv")
adata.write_h5ad("/path/UCART_SLE_8Samples_QC.h5ad")